In [27]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product, chain, combinations
from scipy import stats
from IPython.display import display, HTML
from datetime import datetime
%matplotlib inline

pd.set_option('display.max_rows',100)
display(HTML("<style>div.output_scroll { height: auto; max-height: 72em; }</style>"))

def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

def get_file_name(name):
    return name.replace(':', '-')

In [28]:
inputFile = 'collected-data.csv'
modelFile = ''



dayframes = {
    (0, 5*60+23): 'NIGHT',
    (5*60+24, 5*60+56): 'TWILIGHT',
    (5*60+57, 20*60+33): 'DAYLIGHT',
    (20*60+34, 21*60+6): 'TWILIGHT',
    (21*60+7, 24*60): 'NIGHT'
}

Detected_Activity_List = {
  0.0 : "IN_VEHICLE",
  1.0 : "ON_BICYCLE",
  2.0 : "ON_FOOT",
  3.0 : "STILL",
  4.0 : "UNKOWN",
  5.0 : "TILTING",
  7.0 : "WALKING",
  8.0 : "RUNNING"
}

sensor_type_dict = {
    'MONITORING':-1,
    'INDOOR':-2,
    'GPS_SATELLITES':-3,
    'GPS_FIX_SATELLITES':-4,
    'GPS_FIX':-5,
    'DETECTED_ACTIVITY':-6,
    'WIFI_ACCESS_POINTS':-7,
    'BLUETOOTH_DEVICES':-8
}
dayframes

{(0, 323): 'NIGHT',
 (324, 356): 'TWILIGHT',
 (357, 1233): 'DAYLIGHT',
 (1234, 1266): 'TWILIGHT',
 (1267, 1440): 'NIGHT'}

In [34]:
def preprocess_data(inputFile):
    df = pd.read_csv(inputFile)
    df.loc[:,'FileName'] = inputFile
    for index,row in df.iterrows():
        if  df.loc[index,'sensor_type'] == -1:
            df.loc[index,'sensor_type'] = sensor_type_dict[ row['sensor_name']]
    df_wide = df.pivot_table(index=['FileName','timestamp'], columns='sensor_name', values='value')
    
    last_seen_values = {}
    sensor_types = df['sensor_name'].unique()
    for sensor_type in sensor_types:
        last_seen_values[sensor_type] = float('nan')

    last_gps_fix = -1
    for index,row in df_wide.iterrows():
        if row['MONITORING'] == 0:
            for sensor_type in sensor_types:
                last_seen_values[sensor_type] = float('nan')
        else:
            if row['GPS_FIX'] > 0:
                last_gps_fix = index[1]
            df_wide.loc[index, 'GPS_FIX'] = (index[1] - last_gps_fix)/1000
            for sensor_type in sensor_types:
                if math.isnan(row[sensor_type]):
                    row[sensor_type] = last_seen_values[sensor_type]
                last_seen_values[sensor_type] = row[sensor_type] 

    for index,row in df_wide.iterrows():
        containsNaN = 0.0
        for sensor_type in sensor_types:
            if math.isnan(row[sensor_type]):
                containsNaN = 1.0
                break
        dt = datetime.fromtimestamp(index[1] / 1000)
        minofday = dt.hour * 60 + dt.minute
        for key in dayframes.keys():
            if minofday < key[1] and minofday > key[0]:
                 df_wide.loc[index, 'TIME_OF_DAY'] = dayframes[key]

        df_wide.loc[index,'containsNaN'] = containsNaN

    for value in dayframes.values(): 
        df_wide[value] = df_wide['TIME_OF_DAY'] == value  

    del df_wide['TIME_OF_DAY']

    df_wide = df_wide[(df_wide['MONITORING'] == 1.0) & (df_wide['containsNaN'] == 0.0) & (df_wide['GPS_FIX'] != -1)] 
    del df_wide['containsNaN']

    proximity_name = ""
    light_name = ""
    for sensor_type in sensor_types:
        if 'proximity' in sensor_type.lower():
            proximity_name = sensor_type
            break
        if 'light' in sensor_type.lower():
            light_name = sensor_type

    df_wide.loc[(df_wide[proximity_name] != 0.0),proximity_name] = 1

    for key in Detected_Activity_List.keys(): 
        df_wide[Detected_Activity_List[key]] = df_wide['DETECTED_ACTIVITY'] == key

    df_wide.sort_values(['timestamp'], inplace=True)
    target = df_wide.loc[:,'INDOOR']
    del df_wide['INDOOR']
    del df_wide['MONITORING']
    del df_wide['DETECTED_ACTIVITY']

    cols = [sensor_type for sensor_type in df_wide.columns.to_list() if sensor_type != proximity_name]
    cols.append(proximity_name)
    df_wide = df_wide[cols].replace(True,1.0).replace(False,0.0)
    df_wide.rename(columns={light_name: "LIGHT", proximity_name: "PROXIMITY"}, inplace=True)
    return df_wide

from os import walk,path

dfs = []
root_directory = "datasets"
_, _, filenames = next(walk(root_directory))
for filename in filenames:     
    x = path.join(root_directory,filename)
    dfs.append(preprocess_data(x))
    

df_wide = pd.concat(dfs)
    
df_wide.to_csv('preprocessed_data.csv')
target.to_numpy()
df_wide

sensor_name                                 BLUETOOTH_DEVICES       GPS_FIX  \
FileName                      timestamp                                       
datasets/collected-data-N.csv 1.620724e+12                1.0  1.620724e+09   
                              1.620724e+12                1.0  1.620724e+09   
                              1.620724e+12                1.0  1.620724e+09   
                              1.620724e+12                1.0  1.620724e+09   
                              1.620724e+12                1.0  1.620724e+09   
...                                                       ...           ...   
datasets/collected-data-D.csv 1.620817e+12                2.0  3.098200e+01   
                              1.620817e+12                2.0  3.191300e+01   
                              1.620817e+12                2.0  3.291300e+01   
                              1.620817e+12                2.0  3.391400e+01   
                              1.620817e+12                2.0  3.492500e+01   

sensor_name                                 GPS_FIX_SATELLITES  \
FileName                      timestamp                          
datasets/collected-data-N.csv 1.620724e+12                 0.0   
                              1.620724e+12                 0.0   
                              1.620724e+12                 0.0   
                              1.620724e+12                 0.0   
                              1.620724e+12                 0.0   
...                                                        ...   
datasets/collected-data-D.csv 1.620817e+12                 7.0   
                              1.620817e+12                 7.0   
                              1.620817e+12                 7.0   
                              1.620817e+12                 7.0   
                              1.620817e+12                 7.0   

sensor_name                                 GPS_SATELLITES  LIGHT  \
FileName                      timestamp                             
datasets/collected-data-N.csv 1.620724e+12            16.0   16.0   
                              1.620724e+12            16.0   16.0   
                              1.620724e+12            15.0   16.0   
                              1.620724e+12            16.0   16.0   
                              1.620724e+12            16.0   16.0   
...                                                    ...    ...   
datasets/collected-data-D.csv 1.620817e+12             7.0  235.0   
                              1.620817e+12             7.0  351.0   
                              1.620817e+12             7.0  117.0   
                              1.620817e+12             7.0  161.0   
                              1.620817e+12             7.0  169.0   

sensor_name                                 WIFI_ACCESS_POINTS  NIGHT  \
FileName                      timestamp                                 
datasets/collected-data-N.csv 1.620724e+12                 9.0    0.0   
                              1.620724e+12                 9.0    0.0   
                              1.620724e+12                 9.0    0.0   
                              1.620724e+12                 9.0    0.0   
                              1.620724e+12                 9.0    0.0   
...                                                        ...    ...   
datasets/collected-data-D.csv 1.620817e+12                 1.0    0.0   
                              1.620817e+12                 1.0    0.0   
                              1.620817e+12                 1.0    0.0   
                              1.620817e+12                 1.0    0.0   
                              1.620817e+12                 1.0    0.0   

sensor_name                                 TWILIGHT  DAYLIGHT  IN_VEHICLE  \
FileName                      timestamp                                      
datasets/collected-data-N.csv 1.620724e+12       0.0       1.0         0.0   
                              1

In [22]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_wide.to_numpy(), target, test_size=0.33) 
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)


(1601, 18) (790, 18) (1601,) (790,)


In [23]:
D = X_train.shape[1]
X_train

array([[  4.   ,  64.494,   0.   , ...,   0.   ,   0.   ,   1.   ],
       [  2.   ,  50.103,   0.   , ...,   0.   ,   0.   ,   1.   ],
       [  0.   ,  25.209,   0.   , ...,   0.   ,   0.   ,   1.   ],
       ...,
       [  0.   ,  27.347,   0.   , ...,   0.   ,   0.   ,   1.   ],
       [  0.   ,  45.561,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.   , 106.283,   0.   , ...,   0.   ,   0.   ,   0.   ]])

In [28]:
# Now all the fun Tensorflow stuff
# Build the model


i = tf.keras.layers.Input(shape=(D,),name='Input')
n_i = tf.keras.layers.LayerNormalization(axis=-1,name='Normalization')(i[:,:5])
x = tf.keras.layers.Concatenate(name='Concatenate')([n_i,i[:,5:]])
x = tf.keras.layers.Dense(1, activation='sigmoid', name='Classifier')(x)
model = tf.keras.models.Model(i, x)


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

r = model.fit(X_train,y_train,validation_data=(X_test, y_test), epochs=200)

print("Train score:", model.evaluate(X_train, y_train))
print("Test score:", model.evaluate(X_test, y_test))

Epoch 1/200
51/51 [==============================] - 2s 12ms/step - loss: 0.9658 - accuracy: 0.4066 - val_loss: 0.8537 - val_accuracy: 0.3342
Epoch 2/200
51/51 [==============================] - 0s 6ms/step - loss: 0.8267 - accuracy: 0.3623 - val_loss: 0.7614 - val_accuracy: 0.4468
Epoch 3/200
51/51 [==============================] - 0s 5ms/step - loss: 0.7464 - accuracy: 0.4816 - val_loss: 0.6959 - val_accuracy: 0.5570
Epoch 4/200
51/51 [==============================] - 0s 5ms/step - loss: 0.6846 - accuracy: 0.5628 - val_loss: 0.6414 - val_accuracy: 0.6152
Epoch 5/200
51/51 [==============================] - 0s 5ms/step - loss: 0.6329 - accuracy: 0.6096 - val_loss: 0.5968 - val_accuracy: 0.6759
Epoch 6/200
51/51 [==============================] - 0s 5ms/step - loss: 0.5903 - accuracy: 0.6796 - val_loss: 0.5595 - val_accuracy: 0.7405
Epoch 7/200
51/51 [==============================] - 0s 4ms/step - loss: 0.5541 - accuracy: 0.7439 - val_loss: 0.5277 - val_accuracy: 0.8013
Epoch 8/200


Epoch 59/200
51/51 [==============================] - 0s 3ms/step - loss: 0.2852 - accuracy: 0.9044 - val_loss: 0.2753 - val_accuracy: 0.9038
Epoch 60/200
51/51 [==============================] - 0s 4ms/step - loss: 0.2830 - accuracy: 0.9026 - val_loss: 0.2741 - val_accuracy: 0.9025
Epoch 61/200
51/51 [==============================] - 0s 3ms/step - loss: 0.2820 - accuracy: 0.9032 - val_loss: 0.2727 - val_accuracy: 0.9063
Epoch 62/200
51/51 [==============================] - 0s 4ms/step - loss: 0.2810 - accuracy: 0.8994 - val_loss: 0.2718 - val_accuracy: 0.9063
Epoch 63/200
51/51 [==============================] - 0s 5ms/step - loss: 0.2796 - accuracy: 0.9069 - val_loss: 0.2698 - val_accuracy: 0.9114
Epoch 64/200
51/51 [==============================] - 0s 3ms/step - loss: 0.2780 - accuracy: 0.9026 - val_loss: 0.2693 - val_accuracy: 0.9076
Epoch 65/200
51/51 [==============================] - 0s 5ms/step - loss: 0.2776 - accuracy: 0.9069 - val_loss: 0.2677 - val_accuracy: 0.9101
Epoch 

51/51 [==============================] - 0s 3ms/step - loss: 0.2068 - accuracy: 0.9469 - val_loss: 0.1944 - val_accuracy: 0.9557
Epoch 174/200
51/51 [==============================] - 0s 3ms/step - loss: 0.2071 - accuracy: 0.9482 - val_loss: 0.1956 - val_accuracy: 0.9456
Epoch 175/200
51/51 [==============================] - 0s 3ms/step - loss: 0.2063 - accuracy: 0.9475 - val_loss: 0.1944 - val_accuracy: 0.9532
Epoch 176/200
51/51 [==============================] - 0s 3ms/step - loss: 0.2059 - accuracy: 0.9482 - val_loss: 0.1937 - val_accuracy: 0.9557
Epoch 177/200
51/51 [==============================] - 0s 3ms/step - loss: 0.2055 - accuracy: 0.9500 - val_loss: 0.1929 - val_accuracy: 0.9570
Epoch 178/200
51/51 [==============================] - 0s 3ms/step - loss: 0.2051 - accuracy: 0.9500 - val_loss: 0.1927 - val_accuracy: 0.9557
Epoch 179/200
51/51 [==============================] - 0s 4ms/step - loss: 0.2049 - accuracy: 0.9513 - val_loss: 0.1921 - val_accuracy: 0.9570
Epoch 180/200

In [25]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

NameError: name 'r' is not defined

In [ ]:
# Plot the accuracy too
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()

In [ ]:
model.summary()


In [ ]:
for z in zip(df_wide.columns.values,model.layers[0].get_weights()[0]):
  print(z)
